In [1]:
import torch
import mymodel
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pickle
import MeCab
from mecab_split import mecabsplit
tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')


ModuleNotFoundError: No module named 'MeCab'

In [ ]:
content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in open('mydata.txt','r',encoding='utf-8'):
    line = line.strip().split('\t')
    if len(line)>1:
        intent=line[1]
        if intent not in intent_vocab:
            intent_vocab[intent]=intent_idx
            intent_list.append(intent)
            intent_idx +=1
    else:
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            if it not in content_vocab:
                content_vocab[it] = vocab_idx
                vocab_idx +=1
                
                

In [ ]:
cnn = mymodel.CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) # 파일에 나타난 단어수, 의도 갯수(클래스)
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()

In [ ]:
epoch = 10
for e in range(epoch):
    totalloss = 0
    for line in open('mydata.txt','r',encoding='utf-8'):
        line = line.strip().split('\t')
    
        if len(line)> 1:
            target = Variable(torch.LongTensor([intent_vocab[line[1]]]))
            continue

        optimizer.zero_grad()

        cont = []
        line = mecabsplit(tagger,line[0],False)
        for it in line:
            cont.append(content_vocab[it])
        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)

        loss = F.cross_entropy(pred,target)
        totalloss += loss.data[0]
        loss.backward()
        optimizer.step()
    print (totalloss)

In [ ]:
response = []
for line in open('response.txt','r',encoding='utf-8'):
    line=line.strip()
    response.append(line)

In [ ]:
cnn.eval()
for line in open('testdata.txt','r',encoding='utf-8'):
    line = line.strip()
    
    line = mecabsplit(tagger,line,False)
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it]) # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = cnn(cont)
    v,i = torch.max(pred,1) # pred는 (p1, p2) 즉, 클래스별 확률 v: 둘중 큰값 i:큰값 클래스의 인덱스
    
    print('input : ',line)
    print (torch.nn.functional.softmax(pred,dim=-1).data.numpy() )
    print('intent : ',intent_list[int(i)])
    print(response[int(i)])
    print()